In [107]:
import os
os.getcwd()

'C:\\Users\\Abhishek Upadhyay\\jupyter-lab'

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [117]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
from sklearn.metrics import mean_squared_error

tickers =['TSLA','NVDA','GOOGL','AAPL','MMM','AXP','GS'
          ,'CAT','IBM','MCD','CSCO','MSFT','AMZN','FB'
          ,'JPM','JNJ','COST','FAST','EBAY','CERN','CDW' ]

In [118]:
files=[]
for ticker in tickers:
    files.append(ticker)
    data = pdr.get_data_yahoo(ticker, start='2015-02-28', end=date.today())
    pd.to_pickle(data, "./pickle_files/"+ticker+".pkl")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [119]:
real=pd.DataFrame()

rmse=[]
frcst=[]
prv=[]
for ticker in tickers:
    df = pd.read_pickle("./pickle_files/"+str(ticker)+".pkl")
    val=df.filter(['Adj Close'])
    prv.append(val.tail(1).values)
    
    df["Date"]=pd.to_datetime(df.index,format="%Y-%m-%d")
    df.index=df['Date']
    data=df.sort_index(ascending=True,axis=0)
    new_dataset=pd.DataFrame(index=range(0,len(df)),columns=['Date','Adj Close'])
    for i in range(0,len(data)):
        new_dataset["Date"][i]=data['Date'][i]
        new_dataset["Adj Close"][i]=data["Adj Close"][i]
    
    scaler=MinMaxScaler(feature_range=(0,1))
    final_dataset=new_dataset.values
    train_data=final_dataset[0:1400,:]
    valid_data=final_dataset[1400:,:]
    new_dataset.index=new_dataset.Date
    new_dataset.drop("Date",axis=1,inplace=True)
    scaler=MinMaxScaler(feature_range=(0,1))
    scaled_data=scaler.fit_transform(new_dataset)
    
    
    x_train_data,y_train_data=[],[]
    for i in range(60,len(train_data)):
        x_train_data.append(scaled_data[i-60:i,0])
        y_train_data.append(scaled_data[i,0])
    
    x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
    x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))
    
    lstm_model=Sequential()
    lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(1))
    inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
    inputs_data=inputs_data.reshape(-1,1)
    inputs_data=scaler.transform(inputs_data)
    
    lstm_model.compile(loss='mean_squared_error',optimizer='adam')
    lstm_model.fit(x_train_data,y_train_data,epochs=10,batch_size=1,verbose=2)
    
    X_test=[]
    for i in range(60,inputs_data.shape[0]):
        X_test.append(inputs_data[i-60:i,0])
    X_test=np.array(X_test)
    X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
    predicted_price=lstm_model.predict(X_test)
    predicted_price=scaler.inverse_transform(predicted_price)
    
    valid_data=new_dataset[1400:]
    valid_data['Predictions']=predicted_price
    rmse.append(mean_squared_error(valid_data['Adj Close'],valid_data['Predictions']))
    
    last_60=val[-60:].values
    sc_last_60=scaler.transform(last_60)

    test=[]
    test.append(sc_last_60)
    test=np.array(test)
    test=np.reshape(test,(test.shape[0],test.shape[1],1))
    frcst.append(scaler.inverse_transform(lstm_model.predict(test)))

Epoch 1/10
1340/1340 - 16s - loss: 6.7039e-04
Epoch 2/10
1340/1340 - 14s - loss: 3.5085e-04
Epoch 3/10
1340/1340 - 14s - loss: 2.3877e-04
Epoch 4/10
1340/1340 - 15s - loss: 1.5232e-04
Epoch 5/10
1340/1340 - 15s - loss: 1.5973e-04
Epoch 6/10
1340/1340 - 15s - loss: 1.4897e-04
Epoch 7/10
1340/1340 - 15s - loss: 1.2599e-04
Epoch 8/10
1340/1340 - 14s - loss: 1.1318e-04
Epoch 9/10
1340/1340 - 15s - loss: 9.2629e-05
Epoch 10/10
1340/1340 - 15s - loss: 9.2464e-05


<ipython-input-119-19592eb3ce3f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions']=predicted_price


Epoch 1/10
1340/1340 - 16s - loss: 9.6999e-04
Epoch 2/10
1340/1340 - 14s - loss: 5.5386e-04
Epoch 3/10
1340/1340 - 14s - loss: 3.7034e-04
Epoch 4/10
1340/1340 - 14s - loss: 2.6598e-04
Epoch 5/10
1340/1340 - 14s - loss: 2.5955e-04
Epoch 6/10
1340/1340 - 14s - loss: 2.4936e-04
Epoch 7/10
1340/1340 - 14s - loss: 2.3814e-04
Epoch 8/10
1340/1340 - 14s - loss: 2.6619e-04
Epoch 9/10
1340/1340 - 14s - loss: 1.8340e-04
Epoch 10/10
1340/1340 - 14s - loss: 2.2699e-04


<ipython-input-119-19592eb3ce3f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions']=predicted_price


Epoch 1/10
1340/1340 - 16s - loss: 0.0013
Epoch 2/10
1340/1340 - 14s - loss: 4.3688e-04
Epoch 3/10
1340/1340 - 14s - loss: 3.3620e-04
Epoch 4/10
1340/1340 - 14s - loss: 2.8459e-04
Epoch 5/10
1340/1340 - 14s - loss: 2.5639e-04
Epoch 6/10
1340/1340 - 14s - loss: 2.4977e-04
Epoch 7/10
1340/1340 - 14s - loss: 2.5867e-04
Epoch 8/10
1340/1340 - 15s - loss: 2.7063e-04
Epoch 9/10
1340/1340 - 14s - loss: 2.3215e-04
Epoch 10/10
1340/1340 - 14s - loss: 2.3543e-04


...

<ipython-input-119-19592eb3ce3f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions']=predicted_price


Epoch 1/10
1340/1340 - 21s - loss: 0.0038
Epoch 2/10
1340/1340 - 18s - loss: 0.0015
Epoch 3/10
1340/1340 - 18s - loss: 0.0013
Epoch 4/10
1340/1340 - 19s - loss: 0.0011
Epoch 5/10
1340/1340 - 18s - loss: 0.0011
Epoch 6/10
1340/1340 - 19s - loss: 0.0011
Epoch 7/10
1340/1340 - 18s - loss: 0.0011
Epoch 8/10
1340/1340 - 21s - loss: 9.8591e-04
Epoch 9/10
1340/1340 - 19s - loss: 9.8957e-04
Epoch 10/10
1340/1340 - 19s - loss: 9.3387e-04


<ipython-input-119-19592eb3ce3f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions']=predicted_price


Epoch 1/10
1340/1340 - 21s - loss: 0.0019
Epoch 2/10
1340/1340 - 18s - loss: 8.2415e-04
Epoch 3/10
1340/1340 - 15s - loss: 6.8722e-04
Epoch 4/10
1340/1340 - 17s - loss: 5.0187e-04
Epoch 5/10
1340/1340 - 18s - loss: 4.0138e-04
Epoch 6/10
1340/1340 - 18s - loss: 3.2951e-04
Epoch 7/10
1340/1340 - 16s - loss: 3.2829e-04
Epoch 8/10
1340/1340 - 17s - loss: 3.4185e-04
Epoch 9/10
1340/1340 - 16s - loss: 2.9865e-04
Epoch 10/10
1340/1340 - 20s - loss: 3.2324e-04


<ipython-input-119-19592eb3ce3f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Predictions']=predicted_price


In [120]:
real['Stocks']=files
real['rmse']=rmse
real['Actual']=prv
real['Forecst']=frcst

In [116]:
files

['TSLA',
 'NVDA',
 'GOOGL',
 'AAPL',
 'MMM',
 'AXP',
 'GS',
 'CAT',
 'IBM',
 'MCD',
 'CSCO']

In [130]:
import math
real.rmse=real.rmse.apply(lambda x: math.sqrt(x))
#real.Actual=real.Actual.apply(lambda x: x.strip('[]'))
#real.Forecst=real.Forecst.apply(lambda x: x.strip('[]'))
real

,Stocks,rmse,Actual,Forecst
0,TSLA,3.140419,[[693.72998046875]],[[569.125]]
1,NVDA,1.956431,[[514.239990234375]],[[518.1488]]
2,GOOGL,2.538625,[[2050.0]],[[2001.5776]]
3,AAPL,1.350811,[[121.02999877929688]],[[122.84877]]
4,MMM,1.258223,[[184.9199981689453]],[[184.24991]]
5,AXP,1.368129,[[148.5399932861328]],[[145.43907]]
6,GS,1.535786,[[348.80999755859375]],[[351.17834]]
7,CAT,1.572804,[[229.0]],[[222.29059]]
8,IBM,1.228846,[[127.61000061035156]],[[127.40697]]
9,MCD,1.619386,[[212.33999633789062]],[[205.66039]]


In [1]:
#real.Forecst=real.Forecst.apply(lambda x: x.replace('[',''))